In [0]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving data15smaller_0.csv to data15smaller_0 (1).csv


In [0]:
data18smaller = pd.read_csv("data15smaller_0.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})


In [0]:
!pip install pgmpy==0.1.7
from pgmpy.estimators import HillClimbSearch, BicScore, BayesianEstimator
from pgmpy.models import BayesianModel
from sklearn.model_selection import train_test_split
from pgmpy.inference import VariableElimination

  Using cached https://files.pythonhosted.org/packages/d3/2c/e473e54afc9fae58dfa97066ef6709a7e35a1dd1c28c5a3842989322be00/networkx-1.11-py2.py3-none-any.whl
  Found existing installation: networkx 2.2
    Can't uninstall 'networkx'. No files were found to uninstall.


In [0]:
data18train, data18test = train_test_split(data18smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data18train, scoring_method=BicScore(data18train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'residfar'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'violations'), ('bldgclass', 'landuse'), ('bldgclass', 'ownertype'), ('subsidies_flag', 'residfar'), ('subsidies_flag', 'violations'), ('subsidies_flag', 'bldgclass'), ('subsidies_flag', 'affordability_flag'), ('residfar', 'violations'), ('residfar', 'landuse'), ('violations', 'yearbuilt'), ('violations', 'landuse'), ('violations', 'ownertype'), ('violations', 'unitsres'), ('landuse', 'unitsres')]


In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of unitsres:
+-------------+------------------------+-----------------------+------------------------+----------------------+-----------------------+------------------------+-----------------------+------------------------+
| landuse     | landuse(0)             | landuse(0)            | landuse(1)             | landuse(1)           | landuse(2)            | landuse(2)             | landuse(3)            | landuse(3)             |
+-------------+------------------------+-----------------------+------------------------+----------------------+-----------------------+------------------------+-----------------------+------------------------+
| violations  | violations(0)          | violations(1)         | violations(0)          | violations(1)        | violations(0)         | violations(1)          | violations(0)         | violations(1)          |
+-------------+------------------------+-----------------------+------------------------+----------------------+-----------------------+---

In [0]:
from collections import defaultdict
from pgmpy.inference import VariableElimination
def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    count = 0
    for index, data_point in data.iterrows():
        count = count+1
        if count %1000 == 1:
          print count
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')
test_x = data18test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data18test.loc[:,'violations']==predicted_test['violations']).mean()

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001
33001
34001
35001
36001
37001
38001
39001
40001
41001
42001
43001
44001
45001
46001
47001
48001
49001
50001
51001
52001
53001
54001
55001
56001
Out of sample: 0.8878739155785642


In [0]:

train_x = data18train.drop(columns=['violations'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data18train.loc[:,'violations']==predicted_train['violations']).mean()

In [0]:
data18smaller = pd.read_csv("data15smaller_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})


In [0]:
data18train, data18test = train_test_split(data18smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data18train, scoring_method=BicScore(data18train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'subsidies_flag'), ('bldgclass', 'ownertype'), ('bldgclass', 'residfar'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'unit_ratio'), ('bldgclass', 'violations'), ('residfar', 'violations'), ('yearbuilt', 'residfar'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'subsidies_flag'), ('yearbuilt', 'ownertype'), ('unit_ratio', 'residfar'), ('unit_ratio', 'violations'), ('violations', 'subsidies_flag'), ('landuse', 'yearbuilt'), ('landuse', 'bldgclass')]


In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of bldgclass:
+--------------+-----------------------+------------------------+-----------------------+------------------------+
| landuse      | landuse(0)            | landuse(1)             | landuse(2)            | landuse(3)             |
+--------------+-----------------------+------------------------+-----------------------+------------------------+
| bldgclass(0) | 0.2734740383050296    | 1.0661549123087584e-05 | 8.383635144198524e-05 | 3.505328098710039e-05  |
+--------------+-----------------------+------------------------+-----------------------+------------------------+
| bldgclass(1) | 0.7264825565623135    | 1.0661549123087584e-05 | 8.383635144198524e-05 | 3.505328098710039e-05  |
+--------------+-----------------------+------------------------+-----------------------+------------------------+
| bldgclass(2) | 1.085128316423417e-05 | 0.9571618956234341     | 8.383635144198524e-05 | 0.23366517106001122    |
+--------------+-----------------------+----------------------

In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')
test_x = data18test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data18test.loc[:,'violations']==predicted_test['violations']).mean()

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001
33001
34001
35001
36001
37001
38001
39001
40001
41001
42001
43001
44001
45001
46001
47001
48001
49001
50001
51001
52001
53001
54001
55001
56001
Out of sample: 0.8882076471835465
